In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.svm import SVC
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

from sklearn.metrics import confusion_matrix
from keras.layers import Dense, Activation, Dropout
from sklearn.metrics import classification_report
from keras.models import Sequential
from keras.utils import to_categorical
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
import seaborn as sns
import random

from imblearn.under_sampling import RandomUnderSampler
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import average_precision_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import balanced_accuracy_score, roc_auc_score
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import GridSearchCV

from tensorflow.keras import backend as K
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import balanced_accuracy_score, roc_auc_score
from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import LeaveOneGroupOut
import joblib

In [2]:
df1 = pd.read_csv('/Users/ohidabinteamin/Documents/Stress Prediction Project Three Datasets/StudentLife/week 01/Android_Activity/raw_activity_features_studentlife.csv')
df1 = df1.rename(columns={'Date': 'date'})
df1 = df1.drop('Unnamed: 0', axis=1)
df2 = pd.read_csv('/Users/ohidabinteamin/Documents/Stress Prediction Project Three Datasets/StudentLife/week 01/Stress/recreating_dailystress_features.csv')
df2 = df2.drop('Unnamed: 0', axis=1)

In [3]:
df = pd.merge(df1, df2, on=['uid', 'date'])
print(df.columns)

df = df.sort_values(by='date')

Index(['uid', 'date', 'stationary_duration_morning',
       'walking_duration_morning', 'running_duration_morning',
       'unknown_duration_morning', 'stationary_duration_afternoon',
       'walking_duration_afternoon', 'running_duration_afternoon',
       'unknown_duration_afternoon', 'stationary_duration_evening',
       'walking_duration_evening', 'running_duration_evening',
       'unknown_duration_evening', 'stationary_duration_night',
       'walking_duration_night', 'running_duration_night',
       'unknown_duration_night', 'stress_ratings'],
      dtype='object')


In [4]:
df.isnull().sum()

uid                              0
date                             0
stationary_duration_morning      0
walking_duration_morning         0
running_duration_morning         0
unknown_duration_morning         0
stationary_duration_afternoon    0
walking_duration_afternoon       0
running_duration_afternoon       0
unknown_duration_afternoon       0
stationary_duration_evening      0
walking_duration_evening         0
running_duration_evening         0
unknown_duration_evening         0
stationary_duration_night        0
walking_duration_night           0
running_duration_night           0
unknown_duration_night           0
stress_ratings                   0
dtype: int64

In [5]:
df = df.dropna()
print(len(df))

1216


In [6]:
df.columns

Index(['uid', 'date', 'stationary_duration_morning',
       'walking_duration_morning', 'running_duration_morning',
       'unknown_duration_morning', 'stationary_duration_afternoon',
       'walking_duration_afternoon', 'running_duration_afternoon',
       'unknown_duration_afternoon', 'stationary_duration_evening',
       'walking_duration_evening', 'running_duration_evening',
       'unknown_duration_evening', 'stationary_duration_night',
       'walking_duration_night', 'running_duration_night',
       'unknown_duration_night', 'stress_ratings'],
      dtype='object')

In [7]:
df['stress_ratings'].value_counts()

stress_ratings
medium stress    484
low stress       366
high stress      366
Name: count, dtype: int64

In [8]:
binary_lh_data = df[df['stress_ratings'].isin(['low stress', 'high stress'])]

In [9]:
X = binary_lh_data.drop(columns=['stress_ratings', 'uid', 'date'])
y = binary_lh_data['stress_ratings']
groups = binary_lh_data['uid']

stress_map = {'low stress': 0, 'high stress': 1}
y_encoded = y.map(stress_map).values 

In [10]:
logo = LeaveOneGroupOut()

best_thresholds = []
balanced_accs = []
auc_scores = []

In [11]:
seed_value = 42
np.random.seed(seed_value)
random.seed(seed_value)
tf.random.set_seed(seed_value)

In [12]:
num_splits = len(np.unique(groups))
logo = LeaveOneGroupOut()

In [13]:
num_splits = len(np.unique(groups))
class_weights = compute_class_weight('balanced', classes=np.unique(y_encoded), y=y_encoded)
class_weight_dict = dict(enumerate(class_weights))

param_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': ['scale', 'auto', 0.1, 0.01, 0.001],
    'class_weight': [class_weight_dict]
}

best_thresholds = []
balanced_accs = []
auc_scores = []
auprc_scores = []
best_params_list = []

In [14]:
with tqdm(total=num_splits, desc="LOSO CV Progress", unit="fold", bar_format="{l_bar}{bar} | {n_fmt}/{total_fmt} [{elapsed}<{remaining}] {percentage:3.0f}%") as pbar:
    for train_idx, test_idx in logo.split(X, y_encoded, groups=groups):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y_encoded[train_idx], y_encoded[test_idx]  

        print(f"Train shape: {X_train.shape}, {y_train.shape}")

        scaler = StandardScaler()
        X_train_normalized = scaler.fit_transform(X_train)
        X_test_normalized = scaler.transform(X_test)

        print(f"Normalized shapes: Train {X_train_normalized.shape}, Test {X_test_normalized.shape}")

        print('Performing Grid Search for SVM')
        grid_search = GridSearchCV(
            SVC(kernel='rbf', probability=True),
            param_grid,
            cv=5,
            scoring='balanced_accuracy',
            n_jobs=-1 
        )

        grid_search.fit(X_train_normalized, y_train)
        model_svm_rbf = grid_search.best_estimator_
        
        print("Best parameters:", grid_search.best_params_)
        print("Best cross-validation score:", grid_search.best_score_)
        best_params_list.append(grid_search.best_params_)

        y_test_pred_proba = model_svm_rbf.predict_proba(X_test_normalized)[:, 1]  
        thresholds = np.arange(0.01, 1.0, 0.01)
        best_threshold = max(thresholds, key=lambda t: balanced_accuracy_score(y_test, (y_test_pred_proba > t).astype(int)))

        if len(np.unique(y_test)) > 1:
            auc_score = roc_auc_score(y_test, y_test_pred_proba)
            auprc = average_precision_score(y_test, y_test_pred_proba)
            auc_scores.append(auc_score)
            auprc_scores.append(auprc)
            print(f"AUC Score: {auc_score}")
            print(f"AUPRC Score: {auprc}")
        else:
            auc_scores.append(None)
            auprc_scores.append(None)
            print(f"Skipping AUC and AUPRC computation for this fold as y_test contains only one class: {np.unique(y_test)}")

        y_test_pred_binary = (y_test_pred_proba > best_threshold).astype(int)
        balanced_acc = balanced_accuracy_score(y_test, y_test_pred_binary)
        balanced_accs.append(balanced_acc)
        best_thresholds.append(best_threshold)
        
        print(f"Balanced Accuracy: {balanced_acc}")
        print(f"Best Threshold: {best_threshold}")

        pbar.update(1)

LOSO CV Progress:   0%|                                    | 0/46 [00:00<?]   0%

Train shape: (707, 16), (707,)
Normalized shapes: Train (707, 16), Test (25, 16)
Performing Grid Search for SVM


LOSO CV Progress:   2%|▋                               | 1/46 [00:01<01:05]   2%

Best parameters: {'C': 1, 'class_weight': {0: 1.0, 1: 1.0}, 'gamma': 'auto'}
Best cross-validation score: 0.5634607645875251
AUC Score: 0.6794871794871795
AUPRC Score: 0.6435466494290024
Balanced Accuracy: 0.6762820512820513
Best Threshold: 0.46
Train shape: (718, 16), (718,)
Normalized shapes: Train (718, 16), Test (14, 16)
Performing Grid Search for SVM


LOSO CV Progress:   4%|█▎                              | 2/46 [00:02<00:46]   4%

...

Best parameters: {'C': 1, 'class_weight': {0: 1.0, 1: 1.0}, 'gamma': 'auto'}
Best cross-validation score: 0.5700825241302889
AUC Score: 0.8319327731092437
AUPRC Score: 0.9243037262538542
Balanced Accuracy: 0.8403361344537814
Best Threshold: 0.41000000000000003
Train shape: (706, 16), (706,)
Normalized shapes: Train (706, 16), Test (26, 16)
Performing Grid Search for SVM


LOSO CV Progress:  98%|█████████████████████████████▎ | 45/46 [01:02<00:01]  98%

Best parameters: {'C': 0.1, 'class_weight': {0: 1.0, 1: 1.0}, 'gamma': 0.1}
Best cross-validation score: 0.5775327812284334
AUC Score: 0.2583333333333333
AUPRC Score: 0.6671686665344796
Balanced Accuracy: 0.5
Best Threshold: 0.01
Train shape: (678, 16), (678,)
Normalized shapes: Train (678, 16), Test (54, 16)
Performing Grid Search for SVM


LOSO CV Progress: 100%|██████████████████████████████ | 46/46 [01:02<00:00] 100%

Best parameters: {'C': 100, 'class_weight': {0: 1.0, 1: 1.0}, 'gamma': 'auto'}
Best cross-validation score: 0.5613126354279523
AUC Score: 0.5898520084566596
AUPRC Score: 0.3038687994860978
Balanced Accuracy: 0.645877378435518
Best Threshold: 0.54


In [15]:
print('SVM with Radial basis function (RBF) kernel')
print('Median, 25th Percentile, 75th Percentile: ')

if auc_scores:
    auc_scores_valid = [score for score in auc_scores if score is not None] 

    auc_median = np.median(auc_scores_valid)
    auc_25_percentile = np.percentile(auc_scores_valid, 25)
    auc_75_percentile = np.percentile(auc_scores_valid, 75)
    print(f"AUC Score - Median: {auc_median}, 25th Percentile: {auc_25_percentile}, 75th Percentile: {auc_75_percentile}")

if auprc_scores:
    auprc_scores_valid = [score for score in auprc_scores if score is not None] 

    auprc_median = np.median(auprc_scores_valid)
    auprc_25_percentile = np.percentile(auprc_scores_valid, 25)
    auprc_75_percentile = np.percentile(auprc_scores_valid, 75)
    print(f"AUPRC Score - Median: {auprc_median}, 25th Percentile: {auprc_25_percentile}, 75th Percentile: {auprc_75_percentile}")

balanced_acc_median = np.median(balanced_accs)
balanced_acc_25_percentile = np.percentile(balanced_accs, 25)
balanced_acc_75_percentile = np.percentile(balanced_accs, 75)

print(f"Balanced Accuracy - Median: {balanced_acc_median}, 25th Percentile: {balanced_acc_25_percentile}, 75th Percentile: {balanced_acc_75_percentile}")

SVM with Radial basis function (RBF) kernel
Median, 25th Percentile, 75th Percentile: 
AUC Score - Median: 0.5590277777777778, 25th Percentile: 0.39166666666666666, 75th Percentile: 0.6967032967032967
AUPRC Score - Median: 0.6412177691589456, 25th Percentile: 0.48883348677466326, 75th Percentile: 0.8711290734728235
Balanced Accuracy - Median: 0.679317496229261, 25th Percentile: 0.5852272727272727, 75th Percentile: 0.8396271008403361
